# Tutorial 12-02 - SQLite GeoDatabases

In this tutorial, we'll be joining some tabular and spatial data together.  One of our colleagues has given us a CSV of population and housing data summarized at the census block group level.  They would like to see that data summarized and visualized at the county level.  We'll be using a SQLite to do this.

## Set up the Database and Load Spatial Data

To get things started, you'll need to create a SQLite database.  This is going to serve as the local file where you can combine and manipulate data.  Once you import some packages, you can very easily create a database to work with.

#### 1.  Import packages.

You'll start here by importing some packages to work with.  This is what we're going to use them for:
 - **arcpy** - for creating and copying feature classes
 - **sqlite3** - for direct SQL access to the database
 - **pandas** - for reading from and writing to the database
 - **os** - for handling file paths
 - **arcgis** - for reading a spatially enabled DataFrame

In [1]:
import arcpy
import sqlite3
import pandas as pd
import os
import arcgis

#### 2.  Create a SQLite database.

You're going to use ArcPy to create a SQLite geodatabase here.  That's not strictly necessary.  You could use the `sqlite3` package to create an empty database very easily (by just writing `sqlite3.connect(DB_PATH)`.  You're going to use ArcPy in this case so that you get some extras that turn a standard SQLite database into a geodatabase.

The `CreateSQLiteDatabase` geoprocessing tool is capable of making SQLite geodatabases that conform with the ST Geometry, SpatiaLite, or GeoPackage standards.  In this case, you're just going to use ST Geometry, but you could do either of the other two if your use case required it. 

In [5]:
folder = r'C:\Users\dav11274\Desktop\github\Top-20-Python\Exercises\Chapter 12 - Interacting with Databases'

In [6]:
sl_db = arcpy.management.CreateSQLiteDatabase(
    out_database_name = os.path.join(folder, 'data'),
    spatial_type = "ST_GEOMETRY"
)

In [7]:
sl_db_path = sl_db[0]

#### 3. Copy a feature class into the database.

Now we'll copy a feature class into the geodatabase.  Because we used ArcPy to create this geodatabase, we can use ArcPy to write feature classes into it.

In [8]:
fc_counties = arcpy.conversion.ExportFeatures(
    r"..\Chapter 03 - ArcPy Basics\Chapter 03 Files\Chapter 02 - Working with Maps.gdb\Counties",
    os.path.join(sl_db_path, "Counties")
)

## Explore the GeoDatabase

Now that you've created the geodatabase and put a feature class inside it, it's worth checking out some of the properties of the geodatabase.  Here, we'll look at some of the high-level contents with **ArcPy** and then access some of the underlying tables using the **sqlite3** package.

#### 1.  Walk the geodatabase.

Just like in our previous tutorial, you can walk this geodatabase.  In the following cell, create a `Walk` object and print out all the feature classes and tables.

In [10]:
for root, dirs, files in arcpy.da.Walk(sl_db):
    for dir in dirs:
        print(dir)
    for file in files:
        print(file)

main.Counties


Note that the only feature class or table that is printed is "main.Counties".  This is the feature class that we loaded in the previous step.  The prefix "main." indicates the name of the *schema* in the database.  By default, all the data you put in is going to go into the "main" schema.  You can create and use other schemas to better organize and control your data, but that's not necessary in this case.

#### 2.  Connect to the database using sqlite3.

Now that you've found out what you can about this database using ArcPy, you can use different packages to access the same data.  This is one of the best parts about using a local SQLite database.  It's accessible through many different packages.

Start by using the `sqlite3.connect()` function to create a database connection object.

In [11]:
sl_conn = sqlite3.connect(sl_db_path)
sl_conn

#### 3.  Create a cursor.

A cursor is going to be your primary point of access to execute raw SQL commands on the database.  This concept is consistent across many database platforms.

In [12]:
sl_cursor = sl_conn.cursor()

#### 4.  Execute some SQL.

To start, you can execute a select statement against the "sqlite_master" table.  This table contains an inventory of all the items in the database (including tables, views, indexes, and more).  This will give you a more comprehensive idea of the database contents.

In [13]:
sl_cursor.execute("select * from sqlite_master")

Note that the `.execute()` method only returned the cursor, not the results of the query.  This is common when working with databases.  There are SQL statements where you might not want to actually pull the results into your analysis (like when you're creating a table or a view).  

To fetch the results of your query, you can use a method on the cursor called `fetchall()`.

In [14]:
results = sl_cursor.fetchall()

#### 5.  Explore the results of the query.

This is a very common and basic way to pull data from a database.  Now review the results and see what was returned.

In [20]:
print(f"results type: {type(results)}")
print(f"length of results: {len(results)}")
print(f"first item type: {type(results[0])}")
print(f"first item length: {len(results[0])}")

results type: <class 'list'>
length of results: 15
first item type: <class 'tuple'>
first item length: 5


From this, you can see that the results of your query are a **list** of **tuples**.  This is very common when querying from databases using Python.  Each tuple represents a row of data in the results of your query.  Those rows are all collected in a list.

Now review the first row in the results of your query.

In [21]:
results[0]

('table',
 'st_spatial_reference_systems',
 'st_spatial_reference_systems',
 0,
 'CREATE VIRTUAL TABLE st_spatial_reference_systems USING VSRS()')

This row has five values in it.  You can see there's some descriptive information and a SQL statement.  Without the context of knowing what each column means, it's hard to interpret the results, though.

#### 6.  Use pandas to query.

Using the built-in sqlite3 cursor to query the database is very performant and useful when you understand the data you're looking at, but if you're doing data exploration sometimes it's just easier to use pandas.  Pandas has a method called `read_sql()` that accepts a query and returns a DataFrame.

In [23]:
df_db_objects = pd.read_sql("select * from sqlite_master", sl_conn)
df_db_objects

,type,name,tbl_name,rootpage,sql
0,table,st_spatial_reference_systems,st_spatial_reference_systems,0,CREATE VIRTUAL TABLE st_spatial_reference_syst...
1,table,st_aux_spatial_reference_systems,st_aux_spatial_reference_systems,2,CREATE TABLE st_aux_spatial_reference_systems ...
2,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,st_geometry_columns,st_geometry_columns,4,CREATE TABLE st_geometry_columns (gcid ...
4,index,sqlite_autoindex_st_geometry_columns_1,st_geometry_columns,5,None
5,table,st_vtspindex_interface,st_vtspindex_interface,0,CREATE VIRTUAL TABLE st_vtspindex_interface US...
6,table,Counties,Counties,6,CREATE TABLE [Counties] (OBJECTID integer prim...
7,table,st_spindex__Counties_Shape,st_spindex__Counties_Shape,0,CREATE VIRTUAL TABLE [st_spindex__Counties_Sha...
8,table,st_spindex__Counties_Shape_rowid,st_spindex__Counties_Shape_rowid,379,"CREATE TABLE ""st_spindex__Counties_Shape_rowid..."
9,table,st_spindex__Counties_Shape_node,st_spindex__Counties_Shape_node,380,"CREATE TABLE ""st_spindex__Counties_Shape_node""..."


This gives a bit more context about the data than just using the cursor.  Now you have column names and some nicer formatting.  Both methods are equally useful.  Sometimes you might want to use pandas for data exploration as opposed to just sqlite3 for production data processing (without the pandas overhead).

## Write New Tables to the Database

In addition to reading from the database, you can also use either of the sqlite3 or pandas packages to write to the database.  In this case, you'll use pandas to write a table.  This is easier syntax for smaller datasets (that can fit in memory).  If your data was larger, you might want to consider using a cursor and inserting your data a row at a time to avoid using all your memory.

#### 1.  Read data from a csv.

Before you can use pandas to write to the database, you'll need to read your data into a DataFrame.  In this case, you've been provided a CSV of the data, so you can use the `read_csv()` function.

In [28]:
# path to csv file
csv_path = r".\block_groups.csv"

# read the csv
df_block_groups = pd.read_csv(csv_path, dtype={'GEOID':str})
df_block_groups

,GEOID,Population,Housing_Units
0,060014001001,1789,784
1,060014001002,1249,611
2,060014002001,1022,464
3,060014002002,979,428
4,060014003001,1123,528
...,...,...,...
25602,060490003003,1171,705
25603,060490003001,690,288
25604,060490003002,880,481
25605,060490004001,619,417


#### 2.  Create the county geoid column.

If you're familiar with census data at all, you've probably seen this GEOID format.  If not, it's worth looking into as a lot of demographic data is stored this way.  What we have here is census block group data, which can be summarized as tracts, counties, or states.  The first two numbers in the GEOID column represent the state that the block group is in.  The next three numbers represent the county that the block group is in.

You can use that information to create a new column to use to join this data with the county data you already put into the database.  Using the string accessor (`.str`) on a column will allow you to slice the string.  In this case, you'll only want the first 5 characters.

In [31]:
df_block_groups['GEOID'].str[:5]

0        06001
1        06001
2        06001
3        06001
4        06001
         ...  
25602    06049
25603    06049
25604    06049
25605    06049
25606    06049
Name: GEOID, Length: 25607, dtype: object

Now you can save that as a new column.

In [32]:
df_block_groups['county_geoid'] = df_block_groups['GEOID'].str[:5]

#### 3.  Write the DataFrame to the database.

Similarly to how pandas reads data from the database, pandas also has a method to write to the database.  You can use that method to create a new table containing the data that you've prepared.

In [34]:
df_block_groups.to_sql(
    name = "BlockGroups", 
    con = sl_conn, 
    index = False
)

25607

This returns the number of records that were written to the database.  That number should be the same as the number of records in the DataFrame.

## Summarize Data in the Database

One of the most powerful functions of the database is to be able to summarize and join data on your local machine.  Where pandas can get bogged down by pulling your data all into memory, SQLite can do much larger summaries because it does it locally on your disk.

#### 1.  Write some SQL and execute with pandas.

You can start by writing a basic select statement that groups and summarizes population and housing units at the county level.

In [35]:
sql = """
SELECT
    county_geoid,
    SUM(Population) as county_population,
    SUM(Housing_Units) as county_housing_units
FROM
    BlockGroups
GROUP BY
    county_geoid

"""

NOTE - Using triple quotes allows you to have multi-line strings.  Python will interpret everything until the next set of triple quotes as part of the same string.  This is very handy for saving SQL statements as strings in Python.

Now you can use pandas to read the results of that SQL statement as a DataFrame.

In [37]:
pd.read_sql(sql, sl_conn).head()

,county_geoid,county_population,county_housing_units
0,06001,2980409,1118574
1,06003,2408,3080
2,06005,80948,37610
3,06007,423264,180266
4,06009,90584,54844


#### 2.  Create a view.

The results of that query were exactly what we wanted.  Now if we want to persist those results in the database, we can create a view.  This is basic SQL functionality that we can just execute with Python.

In [38]:
sql_summary_view = """
CREATE VIEW CountyStats AS
SELECT
    county_geoid,
    SUM(Population) as county_population,
    SUM(Housing_Units) as county_housing_units
FROM
    BlockGroups
GROUP BY
    county_geoid

"""
sl_cursor.execute(sql_summary_view)

Now that you've persisted that query as a view, you can select from that view and save some time in the future.  This view will update whenever the underlying data is updated.  So if you change anything in the BlockGroups table, this view will also update.

In [39]:
pd.read_sql("select * from CountyStats", sl_conn).head()

,county_geoid,county_population,county_housing_units
0,06001,2980409,1118574
1,06003,2408,3080
2,06005,80948,37610
3,06007,423264,180266
4,06009,90584,54844


#### 3.  Join data and create another view.

In the same way you just created a view from tabular data, you can also create a view that uses spatial data.  In the following cell, you'll write the SQL to create a view that joins the county feature class with the summarized view you created in the previous step.

This statement uses aliases for the feature class and table and creates an output using columns (or fields) from both.  Because the "shape" column is included, the output will have a spatial component.

In [43]:
sql_join = """
CREATE VIEW CountyStatsSpatial AS
SELECT
    fc.OBJECTID,
    fc.name,
    tbl.county_population,
    tbl.county_housing_units,
    fc.shape
FROM
    Counties fc
    JOIN
        CountyStats tbl
    ON
        fc.geoid = tbl.county_geoid
"""

In [44]:
sl_cursor.execute(sql_join)

#### 4.  Read the spatial view.

You can read the results of this spatial view as a DataFrame using pandas like any other table in the database.

In [45]:
pd.read_sql("select * from CountyStatsSpatial", sl_conn).head()

,OBJECTID,NAME,county_population,county_housing_units,Shape
0,1,Sierra,3236,2127,b'd\xe1\x93\x04x\x13\x00\x00\x18\x08\x9d\xd6\x...
1,2,Sacramento,1585055,587551,b'd\xe1\x93\x04*\x13\x00\x00\x18\x08\xb5\x8e\x...
2,3,Santa Barbara,448229,158279,b'd\xe1\x93\x04(\x16\x00\x00\x18H\xb8\xe8\xc9\...
3,4,Calaveras,90584,54844,b'd\xe1\x93\x04\x82\x0c\x00\x00\x18\x08\xb5\xd...
4,5,Ventura,843843,293080,b'd\xe1\x93\x04\x01\x08\x00\x00\x18H\x85\xdd\x...


Note that this DataFrame shows the "shape" column as a byte string.  This is the default storage format for geometry in SQLite.  There are several ways of converting it to something useful, but the easiest is probably to just use the ArcGIS API for Python to read the data.

In [50]:
df_out = pd.DataFrame.spatial.from_featureclass(
    os.path.join(sl_db[0],"main.CountyStatsSpatial2")
)
df_out.head()

,OBJECTID,NAME,county_population,county_housing_units,SHAPE
0,1,Sierra,3236,2127,"{""rings"": [[[-120.6555850001476, 39.6935599996..."
1,2,Sacramento,1585055,587551,"{""rings"": [[[-121.18857099955193, 38.714307999..."
2,3,Santa Barbara,448229,158279,"{""rings"": [[[-120.7342920000757, 34.9006889998..."
3,4,Calaveras,90584,54844,"{""rings"": [[[-120.6309330000579, 38.3410999995..."
4,5,Ventura,843843,293080,"{""rings"": [[[-119.3292159999184, 34.2278379996..."


As a bonus, we can display this data on a map.  For a more detailed tutorial on this, revisit the chapter on Jupyter Notebooks.

In [51]:
web_map = arcgis.GIS().map("California")
df_out.spatial.plot(

                colors='coolwarm',
                class_count=10,
                map_widget=web_map,
                renderer_type='c',
                col='county_housing_units',
                line_width=0.1,

)
web_map

MapView(layout=Layout(height='400px', width='100%'))